In [1]:
import numpy as np
import matplotlib.pyplot as plt
from Regler_class_file import PI_controller_class

#importing Druckrohrleitung
import sys
import os
current = os.path.dirname(os.path.realpath('Main_Programm.ipynb'))
parent = os.path.dirname(current)
sys.path.append(parent)
from functions.pressure_conversion import pressure_conversion
from Ausgleichsbecken.Ausgleichsbecken_class_file import Ausgleichsbecken_class
from Turbinen.Turbinen_class_file import Francis_Turbine

In [2]:
# define constants

    # for physics
g                   = 9.81                                          # [m/s²]    gravitational acceleration 
rho                 = 1000.                                         # [kg/m³]   density of water 
pUnit_calc          = 'Pa'                                          # [text]    DO NOT CHANGE! for pressure conversion in print statements and plot labels 
pUnit_conv          = 'mWS'                                         # [text]    for pressure conversion in print statements and plot labels


    # for Turbine
Tur_Q_nenn         = 0.85                                           # [m³/s]    nominal flux of turbine 
Tur_p_nenn         = pressure_conversion(10.6,'bar',pUnit_calc)     # [Pa]      nominal pressure of turbine 
Tur_closingTime    = 90.                                            # [s]       closing time of turbine


    # for PI controller
Con_targetLevel     = 8.                                            # [m]
Con_K_p             = 0.1                                           # [-]       proportional constant of PI controller
Con_T_i             = 10.                                           # [s]       timespan in which a steady state error is corrected by the intergal term
Con_deadbandRange   = 0.05                                          # [m]       Deadband range around targetLevel for which the controller does NOT intervene


    # for pipeline
Pip_length  = (535.+478.)                                           # [m]       length of pipeline
Pip_dia     = 0.9                                                   # [m]       diameter of pipeline
Pip_area    = Pip_dia**2/4*np.pi                                    # [m²]      crossectional area of pipeline
Pip_head    = 105.                                                  # [m]       hydraulic head of pipeline without reservoir
Pip_angle   = np.arcsin(Pip_head/Pip_length)                        # [rad]     elevation angle of pipeline 
Pip_n_seg   = 50                                                    # [-]       number of pipe segments in discretization
Pip_f_D     = 0.014                                                 # [-]       Darcy friction factor
Pip_pw_vel  = 500.                                                  # [m/s]     propagation velocity of the pressure wave (pw) in the given pipeline
    # derivatives of the pipeline constants
Pip_dx      = Pip_length/Pip_n_seg                                  # [m]       length of each pipe segment
Pip_dt      = Pip_dx/Pip_pw_vel                                     # [s]       timestep according to method of characteristics
Pip_nn      = Pip_n_seg+1                                           # [1]       number of nodes
Pip_x_vec   = np.arange(0,Pip_nn,1)*Pip_dx                          # [m]       vector holding the distance of each node from the upstream reservoir along the pipeline
Pip_h_vec   = np.arange(0,Pip_nn,1)*Pip_head/Pip_n_seg              # [m]       vector holding the vertival distance of each node from the upstream reservoir


    # for reservoir
Res_area_base       = 10.                                           # [m²]      total base are of the cuboid reservoir   
Res_area_out        = Pip_area                                      # [m²]      outflux area of the reservoir, given by pipeline area
Res_level_crit_lo   = 0.                                            # [m]       for yet-to-be-implemented warnings
Res_level_crit_hi   = np.inf                                        # [m]       for yet-to-be-implemented warnings
Res_dt_approx       = 1e-3                                          # [s]       approx. timestep of reservoir time evolution to ensure numerical stability (see Res_nt why approx.)
Res_nt              = max(1,int(Pip_dt//Res_dt_approx))             # [1]       number of timesteps of the reservoir time evolution within one timestep of the pipeline
Res_dt              = Pip_dt/Res_nt                                 # [s]       harmonised timestep of reservoir time evolution

    # for general simulation
flux_init       = Tur_Q_nenn/1.1                                    # [m³/s]    initial flux through whole system for steady state initialization  
level_init      = Con_targetLevel                                   # [m]       initial water level in upstream reservoir for steady state initialization
simTime_target  = 600.                                              # [s]       target for total simulation time (will vary slightly to fit with Pip_dt)
nt              = int(simTime_target//Pip_dt)                       # [1]       Number of timesteps of the whole system
t_vec           = np.arange(0,nt+1,1)*Pip_dt                        # [s]       time vector. At each step of t_vec the system parameters are stored


In [3]:
# create objects
offset_pressure = pressure_conversion(Pip_head,'mws',pUnit_calc)

# Upstream reservoir
reservoir = Ausgleichsbecken_class(Res_area_base,Res_area_out,Res_dt,pUnit_conv,Res_level_crit_lo,Res_level_crit_hi,rho)
reservoir.set_steady_state(flux_init,level_init)

# downstream turbine
turbine = Francis_Turbine(Tur_Q_nenn,Tur_p_nenn,Tur_closingTime,Pip_dt,pUnit_conv)
turbine.set_steady_state(flux_init,reservoir.get_current_pressure()+offset_pressure)


# level controll
level_control = PI_controller_class(Con_targetLevel,Con_deadbandRange,Con_K_p,Con_T_i,Pip_dt)
level_control.set_control_variable(turbine.get_current_LA(),display_warning=False)


In [4]:
level_vec       = np.zeros_like(t_vec)
level_vec[0]    = level_init
LA_ist_vec      = np.zeros_like(t_vec)
LA_ist_vec[0]   = turbine.get_current_LA()
LA_soll_vec     = np.zeros_like(t_vec)
LA_soll_vec[0]  = turbine.get_current_LA()
Q_vec           = np.zeros_like(t_vec)
Q_vec[0]        = turbine.get_current_Q()

In [5]:
# time loop

for i in range(nt+1):

    if np.mod(i,1e3) == 0:
        print(t_vec[i])

    if i > 0.1*(nt+1):
        reservoir.set_influx(0.)

    p = reservoir.get_current_pressure()
    level_control.update_control_variable(reservoir.level)
    LA_soll = level_control.get_current_control_variable()
    turbine.update_LA(LA_soll)
    turbine.set_pressure(p+offset_pressure)
    LA_soll_vec[i] = LA_soll
    LA_ist_vec[i]  = turbine.get_current_LA()
    Q_vec[i] = turbine.get_current_Q()

    
    reservoir.set_outflux(Q_vec[i],display_warning=False)

    for it_res in range(Res_nt):
        reservoir.timestep_reservoir_evolution()                                                                                                                                                                        
    level_vec[i] = reservoir.get_current_level()
    
    

0.0
40.52
81.04
121.56
162.08
202.6
243.12
283.64
324.16
364.68
405.2
445.72
486.24
526.76
567.28


In [6]:
%matplotlib qt5
# time loop

# create a figure and subplots to display the velocity and pressure distribution across the pipeline in each pipeline step
fig1,axs1 = plt.subplots(3,1)
axs1[0].set_title('Level')
axs1[0].set_xlabel(r'$t$ [$\mathrm{s}$]')
axs1[0].set_ylabel(r'$h$ [$\mathrm{m}$]')
axs1[0].plot(t_vec,level_vec)
axs1[0].set_ylim([0*level_init,1.5*level_init])
axs1[1].set_title('Flux')
axs1[1].set_xlabel(r'$t$ [$\mathrm{s}$]')
axs1[1].set_ylabel(r'$Q$ [$\mathrm{m} / \mathrm{s}^3$]')
axs1[1].plot(t_vec,Q_vec)
axs1[1].set_ylim([0,2*flux_init])
axs1[2].set_title('LA')
axs1[2].set_xlabel(r'$t$ [$\mathrm{s}$]')
axs1[2].set_ylabel(r'$LA$ [%]')
axs1[2].plot(t_vec,LA_soll_vec)
axs1[2].plot(t_vec,LA_ist_vec)
axs1[2].set_ylim([0,1])
fig1.tight_layout()
fig1.show()
